In [1]:
from src.downloader import *
import numpy as np

In [6]:
fpath = download('https://msbis.parliament.go.th/ewtadmin/ewt/parliament_report/download/vote_detail/20210904112822A5.pdf')
images = convert_to_image(fpath)
images.shape